# Notes on the source code for simulating the decision network (exercise 12)

The full source code is available [here](https://neuronaldynamics-exercises.readthedocs.io/en/latest/_modules/neurodynex/competing_populations/decision_making.html#getting_started). 
This is my attempt to understand how the code works.


## Model Overview

We have three neural population in our model: 
    
- excitatory population 1 : prefer stimuli to the left  
- excitatoy population 2 : prefer stimuli to the right 
- inhibitory population

Their activity $A$ is some function $g$ of the input potential $h$ they receive: 

$$
A_{E,k} = g_E (h_{E,k})
$$

We can write out the input potential to each of the pouplation:
    
\begin{align} 
\tau_E \frac{\mathrm{d}h_{E,1}}{\mathrm{d}t} &= -h_{E, 1} + w_{EE} g_E (h_{E,1}) + w_{EI} g_\text{inh} ( h_\text{inh}) + RI_1 \\
\tau_E \frac{\mathrm{d}h_{E,2}}{\mathrm{d}t} &= -h_{E, 2} + w_{EE} g_E (h_{E,2}) + w_{EI} g_\text{inh} ( h_\text{inh}) + RI_2 \\
\tau_\text{inh} \frac{\mathrm{d}h_{\text{inh}}}{\mathrm{d}t} &= -h_\text{inh} + w_{IE} g_E (h_{E,1}) + w_{IE} g_E ( h_{E,2})
\end{align}

## Inhibitory Population Dynamics

The code for the inhibtiory dynamics is:

In [ ]:
inhib_lif_dynamics = """
    s_NMDA_total : 1  # the post synaptic sum of s. compare with s_NMDA_presyn
    dv/dt = (
    - G_leak_inhib * (v-E_leak_inhib)
    - g_AMPA_excit2inhib * s_AMPA * (v-E_AMPA)
    - g_GABA_inhib2inhib * s_GABA * (v-E_GABA)
    - g_NMDA_excit2inhib * s_NMDA_total * (v-E_NMDA)/(1.0+1.0*exp(-0.062*v/volt)/3.57)
    )/Cm_inhib : volt (unless refractory)
    ds_AMPA/dt = -s_AMPA/tau_AMPA : 1
    ds_GABA/dt = -s_GABA/tau_GABA : 1
"""

We note that there are 3 external factors controlling the dynamics of inhibitory neurons:
 
 - excitation of AMPAR in the inhibitory neurons, via either excitatory populations 
 - excitation of NDMAR in the inhibitory neurons, via either exctiatory populations 
 - recurrent inhibition within the network, mediated by opening GABA (excited inhibitory neurons will activate GABA in other inhihbitory neurons, causing inhibition)

The AMPAR and GABAR contributions both have the general form: 

$$
g \times s \times (v - E)
$$

where: 
    
- $g$ is the conductance of the channel 
- $s$ is the gating variable (ie. proportion of channels open)
- $v - E$ is the driving force of the ions for that channel; ie. how far is the current potential from the resting potential for that channel


Note that the dynamics of NMDAR is slightly different, because of the additional dependence of the Magnesium ion block on voltage, so that there is some extra terms to describe its contribution. (This is very common in neural modelling, basically all of XJ Wang's work uses this equation)

$$
g_\text{NMDAR} \times s_\text{NMDAR} \times (v - E_\text{NMDAR}) / (1 + \exp(-0.062v) / 3.57)
$$


Note that the entire thing is determined by some time constant $\tau_\text{inhib}$, which is denoted as `Cm_inhib` (A reminder that $\tau = RC$, so we are actually dividing by $C = \tau / R$)

I need to further read the documentation on this, but I think the bullet points
```
dv/dt = (
- term1
- term2)
```

is just a minus sign, (so I think I can think of it as there always having those driving force to drive those terms to zero, as in $0 - \text{term1}$)

Also note that we have a leak term: 

$$
g_\text{leak_inhib} \times (v - E_\text{leak_inhib})
$$

Note that for the gating variable, only AMPAR and GABAR are defined here, and they are linear; their differential equations are simply of the form:

$$
\frac{ds}{dt} = -\frac{s}{\tau}
$$

On the other hand, NMDAR dynamics is defined by `s_NMDA_total`, which is defined elsewhere... (I am not fully sure what this means)

**Why is `ds_NMDA/dt` not described in the inhibitory dynamics, but described in the excitatroy dynamics?**

I think the reason for this is that:

 - there are NMDAR projections from excitatory to inhibitory populations 
 - there are also NMDAR projections from excitatory populations to itself (recurrent)
 - But there are no NMDAR projections from inhibitory pouplations; inhibitory populations only have GABA projections to itself and other populations
 
Therefore, when you model the inhibitory population, you only need to take account of the NMDAR activation from excitatory popoluations that synapse on it (which here they take as a sum of the NMDAR projections from all excitatory populations I think), and you don't need to model NMDAR within the inhibitory projections because it doesn't contain any NMDAR projections.

### From differential equations to spikes

Finally, we need to move on from defining the membrane potential of the neural population to describing the firing of the population:

In [ ]:
inhib_pop = NeuronGroup(
    N_Inhib, model=inhib_lif_dynamics,
    threshold="v>v_spike_thr_inhib", reset="v=v_reset_inhib", 
    refractory=t_abs_refract_inhib,
    method="rk2")
# initialize with random voltages:
inhib_pop.v = rnd.uniform(v_spike_thr_inhib / b2.mV - 4., 
                          high=v_spike_thr_inhib / b2.mV - 1., size=N_Inhib) * b2.mV

This is relatively simple:

- we specify the number of enurons in `N_Inhib`
- we specify the model that describe the dynamics of the membrane potential of the population
- we define some spiking threshold 
- we define some reset potential after each spike
- as well as a refractory period 
- a more technical point is the specification of the integration method. Here, `rk2` stands for the Runge-Kutta Method for numerical integration. But I am not sure what it actually does.

## Exctitatory population dynamics

Notice that there are a few extra terms in the excitatory population compared to the inhibitory population: 

In [ ]:
excit_lif_dynamics = """
    s_NMDA_total : 1  # the post synaptic sum of s. compare with s_NMDA_presyn
    dv/dt = (
    - G_leak_excit * (v-E_leak_excit)
    - g_AMPA_excit2excit * s_AMPA * (v-E_AMPA)
    - g_GABA_inhib2excit * s_GABA * (v-E_GABA)
    - g_NMDA_excit2excit * s_NMDA_total * (v-E_NMDA)/(1.0+1.0*exp(-0.062*v/volt)/3.57)
    )/Cm_excit : volt (unless refractory)
    ds_AMPA/dt = -s_AMPA/tau_AMPA : 1
    ds_GABA/dt = -s_GABA/tau_GABA : 1
    ds_NMDA/dt = -s_NMDA/tau_NMDA_s + alpha_NMDA * x * (1-s_NMDA) : 1
    dx/dt = -x/tau_NMDA_x : 1
"""

The $dv/dt$ equation is basically the same, but we have additionally defined the decay of the NMDAR dynamics

$$
\frac{d s_\text{NMDAR}}{dt} = -\frac{s_\text{NMDAR}}{\tau} + \alpha x (1 - s) 
$$


where $x$ is some 'intermediate gating variable' (I am actually not sure what it does) that decays over time:

$$
\frac{dx}{dt} = -\frac{x}{\tau_x}
$$

From the neuronal dynamics texbook, it seems that $x$ is an activation term for NMDAR, ie. NMDAR takes some time to open and this term models that. This gives NMDAR a smooth rise in the proportion of NMDAR activated. (Where as $\tau$ in the equation describes the decay of NMDAR synaptic conductance)

See [chapter 3](https://neuronaldynamics.epfl.ch/online/Ch3.S1.html) of the neuronal dynamics textbook. Which says that the tiem constant $\tau_\text{rise}$ (which is $\tau_x$ here) characterises the rise time of the synaptic conductance). 


When there is pre-syanptic input to the population, $x$ is set to 1, which then decay over time. (This model introduce some delay to that). This can be seen in:

In [ ]:
# set a self-recurrent synapse to introduce a delay when updating the intermediate
# gating variable x
syn_x_A2A = Synapses(excit_pop_A, excit_pop_A, on_pre="x += 1.", delay=0.5 * b2.ms)
syn_x_A2A.connect(j="i")
syn_x_B2B = Synapses(excit_pop_B, excit_pop_B, on_pre="x += 1.", delay=0.5 * b2.ms)
syn_x_B2B.connect(j="i")
syn_x_Z2Z = Synapses(excit_pop_Z, excit_pop_Z, on_pre="x += 1.", delay=0.5 * b2.ms)
syn_x_Z2Z.connect(j="i")

## NMDA Projections

The NMDA projections are only from the excitatory populations, and are described by:

In [ ]:
# NMDA projections FROM EXCITATORY to INHIB, A,B,Z
@network_operation()
def update_nmda_sum():
    sum_sNMDA_A = sum(excit_pop_A.s_NMDA)
    sum_sNMDA_B = sum(excit_pop_B.s_NMDA)
    sum_sNMDA_Z = sum(excit_pop_Z.s_NMDA)
    # note the _ at the end of s_NMDA_total_ disables unit checking
    inhib_pop.s_NMDA_total_ = (1.0 * sum_sNMDA_A + 1.0 * sum_sNMDA_B + 1.0 * sum_sNMDA_Z)
    excit_pop_A.s_NMDA_total_ = (w_pos * sum_sNMDA_A + w_neg * sum_sNMDA_B + w_neg * sum_sNMDA_Z)
    excit_pop_B.s_NMDA_total_ = (w_neg * sum_sNMDA_A + w_pos * sum_sNMDA_B + w_neg * sum_sNMDA_Z)
    excit_pop_Z.s_NMDA_total_ = (1.0 * sum_sNMDA_A + 1.0 * sum_sNMDA_B + 1.0 * sum_sNMDA_Z)

Here we see that:

- the inhibtory population receive NMDAR projections from population A, B, and Z with identical weights 
- excitatory population A receive NMDAR projections from itself, which is positive and controlled by the parameter `w_pos` (effectively defining the recurrent connection weight I think)
- exciatory population A also receive *negatively-weighted* projection from population B and Z (to the same extent). I think this is an effective inhibition term? (but then there is also the explicitly defined inhibtiroy population)

## External Input from a Poisson Group

In the model, all 4 populations receive external neural firing input from a single external neural population with Poisson firing. The influence of the firing (connectivity weight) between this external group is different in inhibtiory and exctiatory populations, but is the same in all excitatory populations:

In [ ]:
# now define the connections:
# projections FROM EXTERNAL POISSON GROUP: ####################################################
poisson2Inhib = PoissonInput(target=inhib_pop, target_var="s_AMPA",
                             N=N_extern, rate=firing_rate_extern, weight=w_ext2inhib)
poisson2A = PoissonInput(target=excit_pop_A, target_var="s_AMPA",
                         N=N_extern, rate=firing_rate_extern, weight=w_ext2excit)

poisson2B = PoissonInput(target=excit_pop_B, target_var="s_AMPA",
                         N=N_extern, rate=firing_rate_extern, weight=w_ext2excit)
poisson2Z = PoissonInput(target=excit_pop_Z, target_var="s_AMPA",
                         N=N_extern, rate=firing_rate_extern, weight=w_ext2excit)